In [258]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from sparse_dot_topn import awesome_cossim_topn
from scipy.sparse import csr_matrix
from typing import List

In [272]:
def cosine_similarity(from_vector: np.ndarray,
                      to_vector: np.ndarray,
                      from_list: List[str],
                      to_list: List[str],
                      to_key_list: List[str],
                      nbest,
                      min_similarity: float = 0) -> pd.DataFrame:
    
    if nbest != None:
        if int(nbest) >  len(to_list):
            raise ValueError('best choice must be less than to_list')
    else:
        nbest = int(1)

    if isinstance(to_vector, np.ndarray):
        to_vector = csr_matrix(to_vector)
    if isinstance(from_vector, np.ndarray):
        from_vector = csr_matrix(from_vector)

    # There is a bug with awesome_cossim_topn that when to_vector and from_vector
    # have the same shape, setting topn to 1 does not work. Apparently, you need
    # to it at least to 2 for it to work

    if int(nbest) <= 1:
        similarity_matrix = awesome_cossim_topn(from_vector, to_vector.T, 2, min_similarity)
    elif int(nbest) > 1:
        similarity_matrix = awesome_cossim_topn(from_vector, to_vector.T, nbest, min_similarity)

    if from_list == to_list:
        similarity_matrix = similarity_matrix.tolil()
        similarity_matrix.setdiag(0.)
        similarity_matrix = similarity_matrix.tocsr()

    if int(nbest) <= 1:
        indices = np.array(similarity_matrix.argmax(axis=1).T).flatten()
        similarity = similarity_matrix.max(axis=1).toarray().T.flatten()
    elif int(nbest) > 1:
        similarity = np.flip(np.take_along_axis(similarity_matrix.toarray(), np.argsort(similarity_matrix.toarray(), axis =1), axis=1) [:,-int(nbest):], axis = 1)
        indices = np.flip(np.argsort(np.array(similarity_matrix.toarray()), axis =1)[:,-int(nbest):], axis = 1)
            
    
    if int(nbest) <= 1:
        matches = [to_list[idx] for idx in indices.flatten()]
        key_matches = [to_key_list[idx] for idx in indices.flatten()]
        matches = pd.DataFrame(np.vstack((from_list, matches, key_matches, similarity)).T, columns=["From", "To", "Key", "Similarity"])
        matches.Similarity = matches.Similarity.astype(float)
        matches.loc[matches.Similarity < 0.001, "To"] = None
        matches.loc[matches.Similarity < 0.001, "Key"] = None
    else:
        matches = [np.array([to_list[idx] for idx in l]) for l in indices] ##In progress
        key_matches = [np.array([to_key_list[idx] for idx in l]) for l in indices] ##In progress
        column = []
        column.append("To")
        for i in range(int(nbest) - 1):
            column.append("BestMatch" + "__" + str(i+1))
        column.append("Key")
        for j in range(int(nbest) - 1):
            column.append("Key" + "__" + str(j+1))
        column.append("Similarity")
        for j in range(int(nbest) - 1):
            column.append("Similarity" + "__" + str(j+1))
            
        matches = pd.concat([pd.DataFrame({'From' : from_list}), pd.DataFrame(np.hstack((matches, key_matches, similarity)), columns= column)], axis =1)
        matches.Similarity = matches.Similarity.astype(float)
        matches.loc[matches.Similarity < 0.001, "To"] = None
        matches.loc[matches.Similarity < 0.001, "Key"] = None
        for i in range(int(nbest) - 1):
            matches.loc[matches.Similarity < 0.001, "BestMatch" + "__" + str(i+1)] = None
            matches.loc[matches.Similarity < 0.001, "Key" + "__" + str(i+1)] = None
        
    return matches



def _create_ngrams(string: str) -> List[str]:
    n_gram_range=(3, 3)
    string = _clean_string(string)
    result = []
    for n in range(n_gram_range[0], n_gram_range[1]+1):
        ngrams = zip(*[string[i:] for i in range(n)])
        ngrams = [''.join(ngram) for ngram in ngrams if ' ' not in ngram]
        result.extend(ngrams)
    return result





def _clean_string(string: str) -> str:
    """ Only keep alphanumerical characters and remove extra spaces """
    string = re.sub(r'[^A-Za-z0-9 ]+', '', string.lower())
    string = re.sub('\s+', ' ', string).strip()
    return string





def polyFuzzLogic(fromKey, fromList, toList, trainCC, date, hour_bucket, nbest = 1):
    """ fromList has only one element, toList has many elements > nbest"""
    vectorizer = TfidfVectorizer(min_df=1, analyzer=_create_ngrams).fit(toList + [fromList])
    X = vectorizer.transform(toList)
    Y = vectorizer.transform([fromList])
    matches = cosine_similarity(Y, X, [fromList], toList, trainCC, nbest)
    tempdf = pd.DataFrame()
    tempdf['high_priority_causing_incidents'] = matches.iloc[:,nbest+1:2*nbest+1].values.tolist()[-1]
    tempdf['target_date'] = nbest*[date]
    tempdf['target_hour_bucket'] = nbest*[hour_bucket]
    tempdf['target_low_priority_incident'] = nbest*[fromKey]
    return tempdf

In [260]:
df = pd.read_excel('Data.xlsx')
print(df.shape)

(141476, 62)


In [261]:
#Remove all Nat Columns in dates 
df['Updated'] = pd.to_datetime(df['Updated'], errors='coerce')
df = df.dropna(subset=['Updated'])
print(df.shape)

(105611, 62)


In [262]:
new = df[['number','Updated', 'description', 'priority']]
new['priority'] = new['priority'].apply(lambda s: str(s).lower())
new['number'] = new.number.apply(lambda s: str(s))
new['description'] = new.description.apply(lambda s: str(s))
new.sample(7)

,number,Updated,description,priority
59716,INC2268714,2019-10-25 03:11:42,Outlook (I have issue with qlick view) - Gener...,low
99897,INC2756098,2020-06-17 21:59:02,Course assessment completed in IE for C2C QMMA...,low
65616,INC2331777,2019-11-20 17:16:07,Hi Derek：\n\n孙永召的直线经理没有收到sharepoint系统发的离职邮件，烦请...,medium
104389,INC2814434,2020-07-17 15:14:42,"I took a course ""Privacy and data protection ...",low
67914,INC2353656,2019-11-29 09:11:01,Inbound Delivery number Request STO#4000563374,low
82865,INC2540533,2020-03-11 09:23:28,"Hi Team,\n\nSFDC User ""Manuela Hübinger-Nilken...",low
17127,INC1823488,2019-03-26 16:44:07,I tried to log in to Salesforce and it says it...,low


In [263]:
def convertDateIntoHourBuckets(date):
    if date.hour <= 3:
        return 1
    elif date.hour <= 6:
        return 2
    elif date.hour <= 9:
        return 3
    elif date.hour <= 12:
        return 4
    elif date.hour <= 15:
        return 5
    elif date.hour <= 18:
        return 6
    elif date.hour <= 21:
        return 7
    else: return 8
    
new['hour_bucket'] = new['Updated'].apply(lambda s: convertDateIntoHourBuckets(s))
new['_date_'] = new['Updated'].apply(lambda s : s.strftime("%Y-%m-%d"))

In [264]:
new.sample(10)

,number,Updated,description,priority,hour_bucket,_date_
103718,INC2805816,2020-07-20 15:57:16,"User 10130749 cannot generate QN report, expor...",medium,5,2020-07-20
95441,INC2698594,2020-05-13 07:17:09,Unable to login into VDI in mybd.capgeminiserv...,medium,3,2020-05-13
72611,INC2399627,2019-12-23 14:43:40,bd2662728 @bd.com\n新疆驿讯商贸有限公司\n买房无下拉菜单 无法选择。\n...,low,5,2019-12-23
27146,INC1918088,2019-05-09 02:06:06,User reported issues with a SAP system.,low,1,2019-05-09
83902,INC2552839,2020-02-28 15:16:25,I have completed the quiz in COURSE 473210 but...,low,5,2020-02-28
100229,INC2760276,2020-06-18 15:31:34,I am unable to verify the course after complet...,low,5,2020-06-18
90561,INC2636839,2020-04-10 17:23:32,QMMA102W Instrument Rework: Material Coordinat...,low,6,2020-04-10
27222,INC1918843,2019-05-08 08:15:28,please route to team CAP-EMEA-ERP-TRACIS\n\nse...,low,3,2019-05-08
72110,INC2395092,2019-12-19 12:45:00,SM12 - Number of lock entries exceeds threshol...,low,4,2019-12-19
6914,INC1710058,2019-02-11 14:27:56,Received email with overdue items but they are...,low,5,2019-02-11


In [265]:
date_list = sorted(list(set(new._date_.values)))
date = date_list[5]
bucket = 3
print(date)

2019-01-08


In [266]:
# From List all high/medium tickets in a specified bucket on specific date
inputdf = new.query(" priority == 'low' ").groupby('_date_').get_group(date).groupby('hour_bucket').get_group(bucket)
inputdf.head() #Do a bucket check to move to next iteration

,number,Updated,description,priority,hour_bucket,_date_
97,INC1643900,2019-01-08 08:12:20,Le document QCL-FC-1691 est bloqué les approba...,low,3,2019-01-08
179,INC1644825,2019-01-08 08:10:32,please route to CAP_EMEA_ERP_TRACIS\n\nrequest...,low,3,2019-01-08
246,INC1645469,2019-01-08 09:10:06,C2C course ALQM102W OR Protocol and Aseptic Te...,low,3,2019-01-08
381,INC1646701,2019-01-08 08:12:36,crm Microsoft Dynamics / OpenText is not working,low,3,2019-01-08
437,INC1647148,2019-01-08 09:15:58,MES - MIDL throwing errors when working with r...,low,3,2019-01-08


In [267]:
#to list = all tickets b/w now and infinitely before (!='B$') filtered by low
prevDays = new.query(" priority != 'low' ")[new._date_ == date_list[4]] ## Previous day
sameDay = new.query(" priority != 'low'")[new['hour_bucket'] < bucket][new._date_ == date] # same day
#futureday = new.query(" priority != 'low'")[new['hour_bucket'] == bucket+1][new._date_ == date]

querydf = pd.concat([prevDays,sameDay]).dropna()
querydf # create exception than < nbest

,number,Updated,description,priority,hour_bucket,_date_
156,INC1644537,2019-01-07 06:12:34,"Hello,\nWould it be possible to attach documen...",medium,2,2019-01-07
174,INC1644639,2019-01-07 14:17:15,E070 Report for BD2 December 2018,medium,5,2019-01-07
257,INC1645536,2019-01-07 09:15:06,"Hi Team,\nPlease delete the files starts with ...",medium,3,2019-01-07
425,INC1646994,2019-01-08 05:44:20,source file data format issue,medium,2,2019-01-08


In [273]:
fromListDict = {}
for x, y in zip(inputdf['number'].values.tolist(), inputdf['description'].values.tolist()):
    fromListDict[x] = y
    
toList = querydf['description'].values.astype('U').tolist()
trainCC = querydf['number'].values.astype('U').tolist()

result = pd.concat(list(map(lambda x: polyFuzzLogic(x[0], x[1], toList, trainCC, date, bucket), fromListDict.items())))

In [274]:
result #Get causing descriptions, causing timestamp

,high_priority_causing_incidents,target_date,target_hour_bucket,target_low_priority_incident
0,INC1644537,2019-01-08,3,INC1643900
0,INC1645536,2019-01-08,3,INC1644825
0,INC1645536,2019-01-08,3,INC1645469
0,INC1644537,2019-01-08,3,INC1646701
0,INC1646994,2019-01-08,3,INC1647148
0,INC1644639,2019-01-08,3,INC1647197
0,INC1644537,2019-01-08,3,INC1647749
0,INC1644639,2019-01-08,3,INC1647825


#Set high threshold/nbest=1, use a forward approach for PIER next 24 hours, make continous date pairs to adjust missing dates in-B/W

In [250]:
#Low Priority Ticket information
new[new['number'].isin(result['low_priority_causing_incidents'])][['number','description', 'Updated', 'hour_bucket']]

,number,description,Updated,hour_bucket
35,INC1642823,Good day\n\nWe received following information ...,2019-01-07 01:59:35,1
70,INC1643404,"Dear,\n \n应该按下面这个配置，目前系统里的配置看起来是很久之前非独立版本的配置清单...",2019-01-07 01:57:34,1
191,INC1644939,"2 accounts for the user ""Gregory Wohlleb"" in S...",2019-01-07 10:10:48,4
215,INC1645170,I use DocuSign and noticed that the system now...,2019-01-07 12:18:53,4
286,INC1645878,I take the following course in C2C - IWP1093-E...,2019-01-07 16:14:51,6
419,INC1646956,Date format is different in a Excel sheet in s...,2019-01-08 01:54:46,1


In [253]:
#Target data
new[new['number'].isin(result['target_high_priority_incident'])][['number','description', 'Updated', 'hour_bucket']]

,number,description,Updated,hour_bucket
401,INC1646865,9907068 needs to be created with reconciliatio...,2019-01-08 08:10:49,3
408,INC1646854,Inventory Quantity Reconciliation Report - Val...,2019-01-08 09:16:13,3


In [257]:
# Target Tickets
new[new.number == 'INC1646865']['description'].values

401    9907068 needs to be created with reconciliatio...
Name: description, dtype: object

In [255]:
# causing Tickets
print(new[new.number == 'INC1644939']['description'].values)
print(new[new.number == 'INC1645878']['description'].values)
print(new[new.number == 'INC1642823']['description'].values)

191    2 accounts for the user "Gregory Wohlleb" in S...
Name: description, dtype: object
286    I take the following course in C2C - IWP1093-E...
Name: description, dtype: object
35    Good day\n\nWe received following information ...
Name: description, dtype: object
